In [1]:
from rfo_core.aws.iam import get_aws_session
import sys
sys.path.append("D:\\FVM\\helpers")
from helpers import Zone_names
from rfo_core.configuration import (aws_key, aws_secret, aws_region_default)
# import pandas as pd

In [2]:
import awswrangler as wr
import boto3 as bt

In [3]:
def aws_get_session(region: str = aws_region_default) -> bt.Session:
    return get_aws_session(aws_key=aws_key, aws_secret=aws_secret, aws_region=region)

In [4]:
session = aws_get_session(region='us-east-1')

In [5]:
table="rfo_load_enriched"

In [6]:
#for importing load data for MIDC
names = tuple(Zone_names().values())
# conditions='''WHERE region IN ('CAISO', 'NWPP', 'SOUTHWEST')
#   AND datatype = 'temperature'
#   AND CAST(datetime AS DATE) BETWEEN DATE '2016-01-01' AND DATE '2025-12-31' '''
conditions=f'''WHERE name IN {names}
    AND datatype = 'RTLOAD'
    AND CAST(datetime AS DATE) BETWEEN DATE '2021-09-01' AND DATE '2025-12-31' '''

In [7]:
#To query all columns
# table_info = wr.catalog.table(
#     database="rfo_analytics", 
#     table=table, 
#     boto3_session=session
# )
# columns = table_info['Column Name'].values

#If you know which columns to query
# columns = ["datetime","datatype","avgvalue","siteid","name","station_name","state","region","location","timezone"] # for weather
columns = ["datetime","name","avgvalue","objectid","yes_objectid","iso","timezone"]
# Build SELECT clause with automatic datetime casting
select_parts = []
for col_name in columns:
    if col_name.lower() == 'datetime':
        select_parts.append(f"CAST({col_name} AS timestamp) as {col_name}")
    else:
        select_parts.append(col_name)

select_clause = ",\n    ".join(select_parts)

# Build and execute query
sql = f"""SELECT 
    {select_clause}
FROM {table}
{conditions}
"""

df = wr.athena.read_sql_query(
    sql=sql,
    database="rfo_analytics",
    boto3_session=session
)

In [8]:
#save df a csv file
df.to_csv(f"../data/RTLoad_MIDC+NW_1Sep21-latest.csv", index=False)

In [14]:
df.head()

,datetime,name,avgvalue,objectid,yes_objectid,iso,timezone
0,2021-09-11 15:00:00,PACW-TAC,2009.0,L000082,10002702774,CAISO,America/Los_Angeles
1,2021-09-11 16:00:00,PACW-TAC,2093.0,L000082,10002702774,CAISO,America/Los_Angeles
2,2021-09-11 17:00:00,PACW-TAC,2145.0,L000082,10002702774,CAISO,America/Los_Angeles
3,2021-09-11 18:00:00,PACW-TAC,2187.0,L000082,10002702774,CAISO,America/Los_Angeles
4,2021-09-11 19:00:00,PACW-TAC,2220.0,L000082,10002702774,CAISO,America/Los_Angeles


In [9]:
pd.to_datetime(df['datetime']).max().strftime('%d%m%Y')

NameError: name 'pd' is not defined

In [ ]:
#print the last 'datetime' in the table
sql = "SELECT MAX(CAST(datetime AS DATE)) as last_datetime FROM "+table
last_datetime = wr.athena.read_sql_query(
    sql=sql,
    database="rfo_analytics",
    boto3_session=session
)
print("Last datetime in the table:", last_datetime['last_datetime'].iloc[0])

In [ ]:
#print unique datatypes
# checking all unique datatypes in the table
sql = "SELECT DISTINCT datatype FROM rfo_load_enriched"
unique_datatypes = wr.athena.read_sql_query(
    sql=sql,
    database="rfo_analytics",
    boto3_session=session
)
print(unique_datatypes['datatype'].tolist())

In [ ]:
# checking all unique names in the table for datatype='EIA930_DEMAND'
sql = "SELECT DISTINCT name FROM rfo_load_enriched WHERE datatype='EIA930_DEMAND'"
unique_names = wr.athena.read_sql_query(
    sql=sql,
    database="rfo_analytics",
    boto3_session=session
)

print(sorted(unique_names['name'].tolist()))

['Arizona Public Service Company', 'Associated Electric Cooperative, Inc.', 'Avista Corporation', 'BPA', 'Balancing Authority of Northern California', 'CAISO', 'California', 'Carolinas', 'Central', 'City of Tacoma, Department of Public Utilities', 'Duke Energy Carolinas', 'Duke Energy Florida, Inc.', 'Duke Energy Progress East', 'Duke Energy Progress West', 'El Paso Electric Company', 'Electric Energy, Inc.', 'Florida', 'Florida Municipal Power Pool', 'Florida Power & Light Co.', 'Gainesville Regional Utilities', 'Homestead, City of', 'Idaho Power Company', 'Imperial Irrigation District', 'JEA', 'LG&E and KU Services Company', 'Los Angeles Department of Water and Power', 'Mid-Atlantic', 'Midwest ISO', 'Midwest Region', 'NE-ISO Total', 'NYISO', 'NaturEner Wind Watch, LLC', 'Nevada Power Company', 'New England', 'New Smyrna Beach, Utilities Commission of', 'New York', 'NorthWestern Corporation', 'Northwest', 'Ohio Valley Electric Corporation', 'PUD No. 1 of Douglas County', 'PUD No. 2 of

In [ ]:
# checking all unique names in the table for datatype='EIA930_DEMAND'
sql = "SELECT DISTINCT name FROM rfo_load_enriched WHERE datatype='RTLOAD'"
unique_names = wr.athena.read_sql_query(
    sql=sql,
    database="rfo_analytics",
    boto3_session=session
)
print(sorted(unique_names['name'].tolist()))

['.Z.CONNECTICUT', '.Z.MAINE', '.Z.NEMASSBOST', '.Z.NEWHAMPSHIRE', '.Z.RHODEISLAND', '.Z.SEMASS', '.Z.VERMONT', '.Z.WCMASS', 'AECO', 'AECO Sub Zone', 'AEP', 'AEPAPT', 'AEPIMP', 'AEPKPT', 'AEPOPT', 'AP', 'ATSI', 'AVRN', 'Arizona Public Service Company', 'Avista Corporation', 'BANCMID', 'BANCRDNG', 'BANCRSVL', 'BANCSMUD', 'BANCWASN', 'BGE', 'BPA', 'Balancing Authority of Northern California', 'CAISO', 'CAPITL', 'CENTRL', 'CNCT', 'COMED', 'Central', 'Central (old)', 'City of Tacoma, Department of Public Utilities', 'DAYTON', 'DEOK', 'DOMINION', 'DPL', 'DPLCO', 'DUNWOOD', 'DUQUESNE', 'EASTON', 'EKPC (PJMISO)', 'East (old)', 'El Paso Electric Company', 'GENESE', 'GPU', 'HUDVL', 'Idaho Power Company', 'JCPL', 'LADWP', 'LONGIL', 'METED', 'MHKVL', 'MID-ATLANTIC REGION', 'MILLWD', 'MWD-TAC', 'Midwest ISO', 'NE-ISO Total', 'NEVP-TAC', 'NORTH (NYISO)', 'NYC', 'NYISO', 'North', 'NorthWestern Corporation', 'OE', 'OVEC', 'PACE-TAC', 'PACW-TAC', 'PAPWR', 'PECO', 'PENELEC', 'PEPCO', 'PEPCO Sub Zone', 